In [116]:
import torch
from torch import hub
import torchvision
import torchsummary
import yaml
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import re

In [117]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

In [118]:
with open("params.yaml", "r") as f:
	params = yaml.safe_load(f)

train_methods = params["train"][f"method_{params['method']}"]
del params["train"]
params["train"] = train_methods

In [119]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [120]:
model = hub.load("pytorch/vision", "resnet18", weights=True)
model = model.to(device)

Using cache found in /home/francesco/.cache/torch/hub/pytorch_vision_main


In [121]:
method = params["method"]

if method == 0:
	model.fc = torch.nn.Linear(512, 2).to(device)

if method == 1:
	model.fc = torch.nn.Sequential(
		torch.nn.Linear(512, 128),
		torch.nn.ReLU(),
		torch.nn.Linear(128, 2),
	).to(device)

elif method == 2:
	model = torch.nn.Sequential(model, torch.nn.Linear(1000, 2).to(device))

else:
	raise ValueError("Method not recognised")

print(model)

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [122]:
# torchsummary.summary(model, (3, 244, 244))

In [123]:
def get_images(folder, class_name, class_val):
	return [
		(os.path.join(folder, class_name, path), class_val)
		for path in os.listdir(os.path.join(folder, class_name))
		if path.endswith(".jpg")
	]

class Dataset(torch.utils.data.Dataset):
	def __init__(self, folder, transform=None):
		self.__transform = transform
		
		cats = get_images(folder, "cats", 0)
		dogs = get_images(folder, "dogs", 1)
		
		self.__data = [*cats, *dogs]


	def __len__(self):
		return len(self.__data)
	
	def __getitem__(self, index):
		img = Image.open(self.__data[index][0])
		return self.__transform(img), self.__data[index][1]

In [124]:
transform = torchvision.transforms.Compose([
	# torchvision.transforms.RandomRotation(15),
	# torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
])

train_set = Dataset("./data/training_set/training_set", transform)
test_set = Dataset("./data/test_set/test_set", transform)

train_set, dev_set = torch.utils.data.random_split(train_set, [.9, .1])

train_set_loader = torch.utils.data.DataLoader(train_set, batch_size = params["batch_size"], shuffle = True)
dev_set_loader = torch.utils.data.DataLoader(dev_set, shuffle = True)
test_set_loader = torch.utils.data.DataLoader(test_set, shuffle = True)

In [125]:
def train(model, device, dataset, optimiz, epoch, costF):
  model.train()

  for batch_idx, (data, target) in enumerate(dataset):
    # Move train data to the training device (likely GPU)
    data = data.to(device)
    target = target.to(device)

    # Reset gradient
    optimiz.zero_grad()

    # Forward propagation
    out = model(data)

    # Loss and backpropagation
    cost = costF(out, target)
    cost.backward()
    optimiz.step()

In [126]:
def test(model, device, dataset, costF, setName):
	# Set model in evaluation mode
	model.eval()

	# Init total loss and correct predictions
	loss = 0.0
	correct_pred = 0

	for id, (data, target) in enumerate(dataset):
		# Move train data to the training device (likely GPU)
		data = data.to(device)
		target = target.to(device)

		# Forward propagation
		out = model(data)

		# Define the model prediction as the class with higher probability
		pred = out.argmax(dim=1, keepdim=True)

		# Try to reshape the arrays into what they already are
		# If there were errors in the setup, this will create errors

		# sanity check
		batch_size = data.shape[0]
		pred = pred.view(batch_size)  # [bs,]
		target = target.view(batch_size)  # [bs,]

		# Sum the loss of all inputs
		loss += costF(out, target, reduction='sum').item()

		# Sum the number of correct predictions
		correct_pred += pred.eq(target).sum().item()
		
		# print(f"Item #{id} of {len(dataset)}")

	# Compute statistics
	num_samples = len(dataset.dataset)
	avg_loss = loss / num_samples
	accuracy = float(correct_pred) / num_samples

	# Print statistics
	#   print(f"{setName} ==> Avg epoch loss: {avg_loss:2.04} - accuracy: {accuracy:2.04}")

	return avg_loss, accuracy

In [127]:
def get_optimizer(type, lr):
	if type == "Adam":
		return torch.optim.Adam(model.parameters(), lr=lr)
	
	# if type == "..."
	#	return torch.optim. ...
	
	params["optimizer"] = "SDG"
	return torch.optim.SGD(model.parameters(), lr = lr, momentum=0.9)

In [128]:
if params["loss"] == "binary cross entropy with logits":
	loss = torch.nn.functional.binary_cross_entropy_with_logits
# elif params["loss"] == "..."
#	loss = torch.nn.functional. ...
else:
	loss = torch.nn.functional.cross_entropy

In [129]:
def freeze_params_condition(train):
	for name, param in model.named_parameters():
		if not train(name):
			param.requires_grad = False 
		else:
			param.requires_grad = True

In [130]:
def freeze_params_method_0(choice):
	if choice == "train all":
		freeze_params_condition(lambda name: True)

	if choice == "train last":
		freeze_params_condition(lambda name: name.startswith("fc."))

In [131]:
def freeze_params_method_1(choice):
	if choice == "train all":
		freeze_params_condition(lambda name: True)

	if choice == "train last two":
		freeze_params_condition(lambda name: name.startswith("fc."))

	if choice == "train last":
		freeze_params_condition(lambda name: name.startswith("fc.2."))

In [132]:
def freeze_params_method_2(choice):
	if choice == "train all":
		freeze_params_condition(lambda name: True)

	if choice == "train last":
		freeze_params_condition(lambda name: name.startswith("1."))

In [133]:
freeze_params = [
	freeze_params_method_0,
	freeze_params_method_1,
	freeze_params_method_2,
]

In [134]:
train_losses = []
dev_losses = []

In [135]:
def training_phase(phase_num, optimiz, epochs):
	for epoch in range(epochs):
		train(model, device, train_set_loader, optimiz, epoch, loss)

		train_loss, train_acc = test(model, device, train_set_loader, loss, "TRAIN")
		dev_loss, dev_acc = test(model, device, dev_set_loader, loss, "DEV")

		train_losses.append(train_loss)
		dev_losses.append(dev_loss)
		epochs_list = [i for i, _ in enumerate(train_losses)]

		print(f"| {phase_num:5} | {epoch:5} |", end="")
		print(f"  {train_loss:6.3f}  | {100*train_acc:6.2f} % |", end="")
		print(f"  {dev_loss:6.3f}  | {100*dev_acc:6.2f} % |")

		plt.figure()
		plt.plot(epochs_list, train_losses, label="Train", marker='o')
		plt.plot(epochs_list, dev_losses, label="Dev", marker='o')

		plt.title("Average loss at epochs")

		plt.xlabel("Epoch")
		plt.xticks(epochs_list)

		plt.ylabel("Loss")
		plt.ylim(bottom=0)

		plt.legend()
		plt.savefig("loss.png")
		plt.close()
		print("+-------+-------+----------+----------+----------+----------+")

In [136]:
for phase in range(params["num_phases"]):
	print("\n\n==================================================")
	print(f"MODEL IN PHASE {phase + 1}")
	print("==================================================")
	freeze_params[params["method"]](params["train"][phase])
	torchsummary.summary(model, (3, 244, 244))



MODEL IN PHASE 1
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,408
       BatchNorm2d-2         [-1, 64, 122, 122]             128
              ReLU-3         [-1, 64, 122, 122]               0
         MaxPool2d-4           [-1, 64, 61, 61]               0
            Conv2d-5           [-1, 64, 61, 61]          36,864
       BatchNorm2d-6           [-1, 64, 61, 61]             128
              ReLU-7           [-1, 64, 61, 61]               0
            Conv2d-8           [-1, 64, 61, 61]          36,864
       BatchNorm2d-9           [-1, 64, 61, 61]             128
             ReLU-10           [-1, 64, 61, 61]               0
       BasicBlock-11           [-1, 64, 61, 61]               0
           Conv2d-12           [-1, 64, 61, 61]          36,864
      BatchNorm2d-13           [-1, 64, 61, 61]             128
             ReLU-14

In [137]:
# PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [138]:
print("==================================================")
print("PARAMETERS")
print("==================================================\n")
for key, val in params.items():
	print(yaml.dump({key: val}))
print("\n")

print("==================================================")
print("TRAINING")
print("==================================================\n")

print("+-------+-------+---------------------+---------------------+")
print("|       |       |      TRAIN set      |       DEV set       |")
print("+ Phase + Epoch +----------+----------+----------+----------+")
print("|       |       | Avg loss | Accuracy | Avg loss | Accuracy |")
print("+-------+-------+----------+----------+----------+----------+")

for phase in range(params["num_phases"]):
	optimiz = get_optimizer(params["optimizer"], params["learning_rates"][phase])
	epochs = params["epochs"][phase]
	freeze_params[params["method"]](params["train"][phase])
	training_phase(phase, optimiz, epochs)

PARAMETERS

Available_methods:
  0: replace the FC layer with a new FC
  1: replace the FC layer with two new FCs
  2: add a FC layer after the last FC, and only train that

method: 2

batch_size: 32

num_phases: 1

learning_rates:
- 0.001

epochs:
- 20

optimizer: SDG

loss: cross entropy

train:
- train last



TRAINING

+-------+-------+---------------------+---------------------+
|       |       |      TRAIN set      |       DEV set       |
+ Phase + Epoch +----------+----------+----------+----------+
|       |       | Avg loss | Accuracy | Avg loss | Accuracy |
+-------+-------+----------+----------+----------+----------+
|     0 |     0 |   0.065  |  97.36 % |   0.060  |  97.88 % |
+-------+-------+----------+----------+----------+----------+
|     0 |     1 |   0.055  |  97.96 % |   0.054  |  97.88 % |
+-------+-------+----------+----------+----------+----------+
|     0 |     2 |   0.056  |  97.82 % |   0.069  |  97.62 % |
+-------+-------+----------+----------+----------+-----